<a href="https://colab.research.google.com/github/omid-sakaki-ghazvini/Practices/blob/main/LSTM_EUR_USD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install Dependencies and Setup

<div style="direction:rtl">
<font color='green' size="5px">
 کتابخانه های مورد نیاز را نصب میکنیم
    </font>
</div>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout

import warnings

warnings.filterwarnings('ignore')

<div style="direction:rtl">
<font color='green' size="5px">
 از لینک زیر دیتاست را دانلود کرده و در پوشه هم مسیر همین ژوپیتر نوت بوک قرار دهید
    </font>
</div>

## https://www.kaggle.com/datasets/omidsakaki1370/euro-and-dollar-currency

# 2. Load Data

<div style="direction:rtl">
<font color='green' size="5px">
توسط خط فرمان زیر، دیتا را فراخوانی میکنیم
    </font>
</div>

In [ ]:
df = pd.read_csv('/kaggle/input/euro-and-dollar-currency/EUR_USD_DATA.csv')
df.head(10)

# 3.Data preparation

In [ ]:
df.info()

In [ ]:
df = df.fillna(0)
l=list(df.keys())
l

In [ ]:
missing_values_count = df.isnull().sum()
missing_values_count

In [ ]:
df = df.drop(columns = ['Date'])

# 4. Split & Scale Data

In [ ]:
df_for_training = df.astype(float)
scalar = StandardScaler()
scalar.fit(df_for_training)
df_for_training_scaled =scalar.transform(df_for_training)
df_for_training_scaled

In [ ]:
y = [x[0] for x in df_for_training_scaled]
split = int(len(df_for_training_scaled)*0.80)
print(split)

In [ ]:
X_train = df_for_training_scaled[:split]
X_test = df_for_training_scaled[split:len(df_for_training_scaled)]
Y_train = y[:split]
Y_test = y[split:len(y)]

print(len(X_train))
print(len(X_test))
print(len(Y_train))
print(len(X_test))

# 5. Model Architecture

In [ ]:
n = 3 # Looking 15 days in the past
Xtrain,Xtest,Ytrain,Ytest = [],[],[],[]

for i in range(n,len(X_train)):
    Xtrain.append(X_train[i-n:i,:X_train.shape[1]])
    Ytrain.append(Y_train[i])  # predicting next record
for i in range(n,len(X_test)):
    Xtest.append(X_test[i-n:i,:X_test.shape[1]])
    Ytest.append(Y_test[i])  # Predicting next record

In [ ]:
val = Ytrain[0]

Xtrain,Ytrain = (np.array(Xtrain),np.array(Ytrain))
Xtest,Ytest = (np.array(Xtest),np.array(Ytest))
Xtrain = np.reshape(Xtrain,(Xtrain.shape[0],Xtrain.shape[1],Xtrain.shape[2]))
Xtest = np.reshape(Xtest,(Xtest.shape[0],Xtest.shape[1],Xtest.shape[2]))

print(Xtrain.shape)
print(Xtest.shape)
print(Ytrain.shape)
print(Ytest.shape)

In [ ]:
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(Xtrain.shape[1],Xtrain.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(50,return_sequences=True,input_shape=(Xtrain.shape[1],Xtrain.shape[2])))
model.add(Dropout(0.4))

model.add(LSTM(50,input_shape=(Xtrain.shape[1],Xtrain.shape[2])))
model.add(Dropout(0.5))

model.add(Dense(1))
model.compile(loss='mse',optimizer='adam',metrics=[tf.keras.metrics.RootMeanSquaredError(),tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanAbsolutePercentageError()])
hist=model.fit(Xtrain,Ytrain,epochs=100,validation_data=(Xtest,Ytest),batch_size=32)

In [ ]:
plt.title('LSTM Loss')
plt.plot(hist.history['loss'],'r-.', label='train')
plt.plot(hist.history['val_loss'],'b-.', label='test')
plt.legend()
plt.show()
plt.title('LSTM root_mean_squared_error')
plt.plot(hist.history['root_mean_squared_error'],'r-.', label='train')
plt.plot(hist.history['val_root_mean_squared_error'],'b-.', label='test')
plt.legend()
plt.show()

plt.title('LSTM mean_absolute_percentage_error')
plt.plot(hist.history['mean_absolute_percentage_error'],'r-.', label='train')
plt.plot(hist.history['val_mean_absolute_percentage_error'],'b-.', label='test')
plt.legend()
plt.show()

# 6.Result

In [ ]:
trainPredict = model.predict(Xtrain)
testPredict = model.predict(Xtest)

[loss, mean_absolute_error, mean_absolute_percentage_error, root_mean_squared_error] = model.evaluate(Xtest, Ytest)
print("Testing set RMSE: ", root_mean_squared_error)

trainPredict = np.c_[trainPredict,
                     np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),
                     np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),
                     np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),
                     np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),
                     np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),
                     np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),
                     np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),
                     np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),
                     np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),
                     np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),
                     np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),
                     np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),
                     np.zeros(trainPredict.shape),np.zeros(trainPredict.shape),np.zeros(trainPredict.shape)]
testPredict = np.c_[testPredict,
                    np.zeros(testPredict.shape),np.zeros(testPredict.shape),np.zeros(testPredict.shape),
                    np.zeros(testPredict.shape),np.zeros(testPredict.shape),np.zeros(testPredict.shape),
                    np.zeros(testPredict.shape),np.zeros(testPredict.shape),np.zeros(testPredict.shape),
                    np.zeros(testPredict.shape),np.zeros(testPredict.shape),np.zeros(testPredict.shape),
                    np.zeros(testPredict.shape),np.zeros(testPredict.shape),np.zeros(testPredict.shape),
                    np.zeros(testPredict.shape),np.zeros(testPredict.shape),np.zeros(testPredict.shape),
                    np.zeros(testPredict.shape),np.zeros(testPredict.shape),np.zeros(testPredict.shape),
                    np.zeros(testPredict.shape),np.zeros(testPredict.shape),np.zeros(testPredict.shape),
                    np.zeros(testPredict.shape),np.zeros(testPredict.shape),np.zeros(testPredict.shape),
                    np.zeros(testPredict.shape),np.zeros(testPredict.shape),np.zeros(testPredict.shape),
                    np.zeros(testPredict.shape),np.zeros(testPredict.shape),np.zeros(testPredict.shape),
                    np.zeros(testPredict.shape),np.zeros(testPredict.shape),np.zeros(testPredict.shape),
                    np.zeros(testPredict.shape),np.zeros(testPredict.shape),np.zeros(testPredict.shape)]
trainPredict = scalar.inverse_transform(trainPredict)
trainPredict = [x[0] for x in trainPredict]
testPredict = scalar.inverse_transform(testPredict)
testPredict = [x[0] for x in testPredict]
Ytrain = np.c_[Ytrain,
               np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),
               np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),
               np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),
               np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),
               np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),
               np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),
               np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),
               np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),
               np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),
               np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),
               np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),
               np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),
               np.zeros(Ytrain.shape),np.zeros(Ytrain.shape),np.zeros(Ytrain.shape)]
Ytest = np.c_[Ytest,
              np.zeros(Ytest.shape),np.zeros(Ytest.shape),np.zeros(Ytest.shape),
              np.zeros(Ytest.shape),np.zeros(Ytest.shape),np.zeros(Ytest.shape),
              np.zeros(Ytest.shape),np.zeros(Ytest.shape),np.zeros(Ytest.shape),
              np.zeros(Ytest.shape),np.zeros(Ytest.shape),np.zeros(Ytest.shape),
              np.zeros(Ytest.shape),np.zeros(Ytest.shape),np.zeros(Ytest.shape),
              np.zeros(Ytest.shape),np.zeros(Ytest.shape),np.zeros(Ytest.shape),
              np.zeros(Ytest.shape),np.zeros(Ytest.shape),np.zeros(Ytest.shape),
              np.zeros(Ytest.shape),np.zeros(Ytest.shape),np.zeros(Ytest.shape),
              np.zeros(Ytest.shape),np.zeros(Ytest.shape),np.zeros(Ytest.shape),
              np.zeros(Ytest.shape),np.zeros(Ytest.shape),np.zeros(Ytest.shape),
              np.zeros(Ytest.shape),np.zeros(Ytest.shape),np.zeros(Ytest.shape),
              np.zeros(Ytest.shape),np.zeros(Ytest.shape),np.zeros(Ytest.shape),
              np.zeros(Ytest.shape),np.zeros(Ytest.shape),np.zeros(Ytest.shape)]
Ytrain = scalar.inverse_transform(Ytrain)
Ytrain =  [x[0] for x in Ytrain]

Ytest = scalar.inverse_transform(Ytest)
Ytest =  [x[0] for x in Ytest]
# Plotting the train presict
plt.figure(figsize=(36,10))
plt.plot(trainPredict,color='blue',label='Prediction')
plt.plot(Ytrain,color='red',label='Actual')
plt.legend()
plt.show